In [1]:
# Own cnn  architecture - mask detection CNN.ipynb

# import packages
import cv2
import os
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np


Using TensorFlow backend.


In [2]:
# load model # Accuracy=97.4 , validation Accuracy = 99.1 # very light model, size =5MB
model = load_model('model/model_acc_958_vacc_986_epoch_50.h5')  # cnn

In [3]:
# model accept below hight and width of the image
img_width, img_hight = 200, 200


In [4]:
# Load the Cascade face Classifier
face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")

In [5]:
# parameters for text
# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (1, 1)
class_lable = ' '
# fontScale
fontScale = 1  # 0.5
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2  # 1

In [7]:
img_path = os.path.join('temp')
img_list = os.listdir(img_path)

In [10]:
for img_name in img_list:
    # read image from webcam
    color_img = cv2.imread('temp/%s' % img_name)

In [11]:
    # resize image with 50 % ratio
    scale = 20
    width = int(color_img.shape[1] * scale / 100)
    height = int(color_img.shape[0] * scale / 100)
    dim = (width, height)
    # resize image
    color_img = cv2.resize(color_img, dim, interpolation=cv2.INTER_AREA)

In [12]:
    # Convert to grayscale
    gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

In [13]:
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)

In [14]:
    # take face then predict class mask or not mask then draw recrangle and text then display image
    img_count = 0
    for (x, y, w, h) in faces:
        org = (x - 10, y - 10)
        img_count += 1
        color_face = color_img[y:y + h, x:x + w]  # color face
        cv2.imwrite('faces/input/%dface.jpg' % (img_count), color_face)
        img = load_img('faces/input/%dface.jpg' % (img_count), target_size=(img_width, img_hight))

        img = img_to_array(img) / 255
        img = np.expand_dims(img, axis=0)
        pred_prob = model.predict(img)
        # print(pred_prob[0][0].round(2))
        pred = np.argmax(pred_prob)

        if pred == 0:
            print("User with mask prediction score = ", pred_prob[0][0])
            class_lable = "Mask"
            color = (255, 0, 0)
            cv2.imwrite('faces/with_mask/%dface.jpg' % (img_count), color_face)
            cv2.rectangle(color_img, (x, y), (x + w, y + h), (0, 0, 255), 3)
            # Using cv2.putText() method
            cv2.putText(color_img, class_lable, org, font,
                        fontScale, color, thickness, cv2.LINE_AA)
            cv2.imwrite('faces/with_mask/%dmask.jpg' % (img_count), color_img)

        else:
            print('user not wearing mask score = ', pred_prob[0][1])
            class_lable = "No Mask"
            color = (0, 255, 0)
            cv2.imwrite('faces/without_mask/%dface.jpg' % (img_count), color_face)
            cv2.imwrite('faces/with_mask/%dno_mask.jpg' % (img_count), color_img)

        cv2.rectangle(color_img, (x, y), (x + w, y + h), (0, 0, 255), 3)
        # Using cv2.putText() method
        cv2.putText(color_img, class_lable, org, font,
                    fontScale, color, thickness, cv2.LINE_AA)

In [ ]:
  # display image
cv2.imshow('LIVE multiimageface mask detection', color_img)

cv2.waitKey()

In [ ]:
# close all windows
cv2.destroyAllWindows()